In [5]:
%pip install chromadb langchain langchain-community langchain-text-splitters pypdf langchain-groq langchain_huggingface transformers sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [6]:
# Environment Setup
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [8]:
# Laoding the document
loader = PyPDFLoader("Documents/Porsche Case File.pdf")
pages = loader.load()

In [9]:
print(pages[0])

page_content='Pooja Gagan Jain vs State Of Maharashtra on 25 June, 2024
Author: Bharati Dangre
Bench: Bharati Dangre
 2024:BHC-AS:24726-DB
                                                                                       WP-2372-2024.doc
           Rajshree
                                        IN THE HIGH COURT OF JUDICATURE AT BOMBAY
                                                 CRIMINAL APPELLATE JURISDICTION
                                            CRIMINAL WRIT PETITION NO.2372 OF 2024
                          Pooja Gagan Jain                   ]    ..   Petitioner
                                           vs.
                          State of Maharashtra               ]    ..   Respondent
                          Mr. Aabad Ponda, Senior Advocate a/w Prashant Patil, Swapnil
                          Ambure, Pranav Patil, Avantika Sharma, Nida Khan, Swati
                          Pandey, Vinayak Patil, Anant Charkhe, Vishal Nevshe and
                          R.B

In [10]:
# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
splits = text_splitter.split_documents(pages)

In [11]:
print(splits[0])

page_content='Pooja Gagan Jain vs State Of Maharashtra on 25 June, 2024
Author: Bharati Dangre
Bench: Bharati Dangre
 2024:BHC-AS:24726-DB
                                                                                       WP-2372-2024.doc
           Rajshree
                                        IN THE HIGH COURT OF JUDICATURE AT BOMBAY
                                                 CRIMINAL APPELLATE JURISDICTION
                                            CRIMINAL WRIT PETITION NO.2372 OF 2024
                          Pooja Gagan Jain                   ]    ..   Petitioner
                                           vs.
                          State of Maharashtra               ]    ..   Respondent
                          Mr. Aabad Ponda, Senior Advocate a/w Prashant Patil, Swapnil
                          Ambure, Pranav Patil, Avantika Sharma, Nida Khan, Swati
                          Pandey, Vinayak Patil, Anant Charkhe, Vishal Nevshe and
                          R.B

In [15]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)

In [16]:
persist_directory = "chroma_db_info_retrieval"

In [21]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [22]:
retriever = vectordb.as_retriever()
print(retriever)

tags=['Chroma', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000017086383F10> search_kwargs={}


In [23]:
# LLM from Groq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

In [24]:
# Create a Q&A chain
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [26]:
# Run the chain
query = "Give me a detailedsummary of the case"
response = chain.invoke({"query": query})

In [28]:
print(response["result"])

The case appears to be a judicial proceeding involving a Child in Conflict with Law (CCL) who was allegedly involved in a fatal accident. Here is a detailed summary of the case:

**Background:**
The CCL, whose age is not specified, had completed his 12th standard examination on May 17, 2024. On the same night, he left his house to attend a late-night party with his friends at a pub. His parents allowed him to go to the pub and consume liquor, and also permitted him to use a Porsche car, which was allegedly not registered as per the rules of the Road Traffic Office.

**The Incident:**
The CCL, who did not have a driving license, drove the unregistered car rashly and negligently, resulting in a fatal accident that caused the death of two people on the spot. The CCL was allegedly under the influence of liquor at the time of the incident.

**Proceedings:**
The CCL was released on bail on May 19, 2024, with certain conditions, including undergoing psychological treatment and counseling. How

In [29]:
print(response["source_documents"][0])

page_content='A specific argument was advanced on behalf of the CCL that he was already released on bail
considering all the necessary aspects under the Act of 2015 and his mental, physical and social
WP-2372-2024.doc health was also impacted in the wake of the incident and his parents are capable
to care for him and to protect him and they have appointed a security team, outside the house to
ensure his safety.
The learned APP focused upon the aspect of amendment of the order, under Section 104 of the Act
and offered a clarification that the application is not intended to revoke the order passed earlier and
the most highlighted aspects of the accident, were placed before the Board. It was also submitted
that the wrong act of the CCL had created apprehensions in the mind of common people, who had
become doubtful about their safety, on the public road.
It was also argued that after the incident, the CCL became a victim of mob lynching and if released
on bail, there is a moral, physical a